# Importing request

In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from config import api_key
from config import api_id
from config import api_key2

# Creating random list for MSAs
Keith wrote the code to generate the random list from the top polluting MSAs

In [2]:
# The data to load
f = "../data/msa.csv"

# Count the lines
num_lines = sum(1 for l in open(f))

# Sample size - retrieving header and 5 MSA's
size = 10

# The row indices to skip - make sure 0 is not included to keep the header
skip_idx = random.sample(range(1, num_lines), num_lines - size)

# Read the data
msa = pd.read_csv(f, skiprows=skip_idx)

# Display the sample
msa


,MSA
0,"Cincinnati-Wilmington-Maysville, OH-KY-IN"
1,"Cleveland-Akron-Canton, OH"
2,"Fresno-Madera-Hanford, CA"
3,"Houston-The Woodlands, TX"
4,"Indianapolis-Carmel-Muncie, IN"
5,"Philadelphia-Reading-Camden, PA-NJ-DE-MD"
6,"Phoenix-Mesa, AZ"
7,"Sacramento-Roseville, CA"
8,"Shreveport-Bossier City-Minden, LA"


# Striping Leading/Trailing Spaces for Merge

In [3]:
msa['MSA1']=msa['MSA'].str.strip()
msa

,MSA,MSA1
0,"Cincinnati-Wilmington-Maysville, OH-KY-IN","Cincinnati-Wilmington-Maysville, OH-KY-IN"
1,"Cleveland-Akron-Canton, OH","Cleveland-Akron-Canton, OH"
2,"Fresno-Madera-Hanford, CA","Fresno-Madera-Hanford, CA"
3,"Houston-The Woodlands, TX","Houston-The Woodlands, TX"
4,"Indianapolis-Carmel-Muncie, IN","Indianapolis-Carmel-Muncie, IN"
5,"Philadelphia-Reading-Camden, PA-NJ-DE-MD","Philadelphia-Reading-Camden, PA-NJ-DE-MD"
6,"Phoenix-Mesa, AZ","Phoenix-Mesa, AZ"
7,"Sacramento-Roseville, CA","Sacramento-Roseville, CA"
8,"Shreveport-Bossier City-Minden, LA","Shreveport-Bossier City-Minden, LA"


In [4]:
str(len(msa))

'9'

# Reading in MSA Crosswalk info for MSA codes

In [5]:
file = "../data/msa_crosswalk.csv"
crosswalk = pd.read_csv(file)
crosswalk1 = crosswalk[['CBSA Code','CSA Title']].sort_values('CSA Title',ascending = False).rename(columns = {'CSA Title':'MSA1'}).dropna().drop_duplicates()
crosswalk1.head()

,CBSA Code,MSA1
1503,41400,"Youngstown-Warren, OH-PA"
1893,49660,"Youngstown-Warren, OH-PA"
1865,48700,"Williamsport-Lock Haven, PA"
974,30820,"Williamsport-Lock Haven, PA"
1860,48620,"Wichita-Arkansas City-Winfield, KS"


# Joining MSA Crosswalk to Top MSAs to get the CBSA Codes needed for API Pulls

In [6]:
msa_codes = pd.merge(crosswalk1,msa,
                how = 'inner',
                on = 'MSA1')

print(msa_codes.head())
print('-----------------------------------------------------------------------')
print("There are multiple CBSA Codes per MSA, there were " + str(len(msa)) +" codes before join and  "+ str(len(msa_codes)) +" CBSA codes after the join.")

  CBSA Code                                      MSA1  \
0     40900                  Sacramento-Roseville, CA   
1     46020                  Sacramento-Roseville, CA   
2     49700                  Sacramento-Roseville, CA   
3     37980  Philadelphia-Reading-Camden, PA-NJ-DE-MD   
4     36140  Philadelphia-Reading-Camden, PA-NJ-DE-MD   

                                         MSA  
0                   Sacramento-Roseville, CA  
1                   Sacramento-Roseville, CA  
2                   Sacramento-Roseville, CA  
3   Philadelphia-Reading-Camden, PA-NJ-DE-MD  
4   Philadelphia-Reading-Camden, PA-NJ-DE-MD  
-----------------------------------------------------------------------
There are multiple CBSA Codes per MSA, there were 9 codes before join and  32 CBSA codes after the join.


# API Call - Pollution Part 1
using a for loop to loop through three years work of PM2.5 data

In [97]:
response_sample = []
start = ["20150101","20140101","20130101"]
end = ["20151231","20141231","20131231"]
codes = msa_codes['CBSA Code']


for index in range(len(start)):
    for each_msa in codes:
        url = f"https://aqs.epa.gov/data/api/sampleData/byCBSA?email={api_id}&key={api_key}&param=88101&bdate={start[index]}&edate={end[index]}&cbsa={each_msa}"
        response_sample.append(requests.get(url).json())

# Pulling Data and putting in list
There are two loops, the first loop is through the 33 different MSAs. Unfortunately the EPA does not always sample every 3 dyas like their website says. This leads to certain sampling sites having different lengths of samples. The second loop goes from 0 to the length of the number of samples they do have

In [115]:
time = []
date = []
cbsa_code = []
site = []
sample = []


for x in range(len(response_sample)):
# for x in range(0,32):
    for y in range(0,response_sample[x]['Header'][0]['rows']):
        time.append(response_sample[x]['Data'][y]['time_local'])
        date.append(response_sample[x]['Data'][y]['date_local'])
        cbsa_code.append(response_sample[x]['Data'][y]['cbsa_code'])
        site.append(response_sample[x]['Data'][y]['site_number'])
        sample.append(response_sample[x]['Data'][y]['sample_measurement'])

columns = ['time','date','cbsa_code','site','sample']
df_sample = pd.DataFrame(data = list(zip(time,date,cbsa_code,site,sample)), columns = columns)

df_sample.head()

,time,date,cbsa_code,site,sample
0,00:00,2015-12-20,40900,0006,5.7
1,00:00,2015-12-08,40900,0006,9.5
2,00:00,2015-11-26,40900,0006,7.3
3,00:00,2015-11-14,40900,0006,16.9
4,00:00,2015-11-02,40900,0006,2.0


# Converting date to DateTime
needed to group by month which is format that Oil Data is in

In [117]:
df_sample['date'] = pd.to_datetime(df_sample.date,format = '%Y-%m')
print(df_sample.head(1))
print('---------------------------------------------------------')
print("There are "+ str(len(df_sample)) +" rows of PM2.5 data")

    time       date cbsa_code  site  sample
0  00:00 2015-12-20     40900  0006     5.7
---------------------------------------------------------
There are 646104 rows of PM2.5 data


In [119]:
df_sample['month_year'] = df_sample['date'].dt.strftime('%m-%Y')
df_sample.head(1)

,time,date,cbsa_code,site,sample,month_year
0,00:00,2015-12-20,40900,0006,5.7,12-2015


converting sample to float to be used for regression/analysis later

In [120]:
df_sample= df_sample.astype({'sample': float})

# Grouping Hourly Data by day & Merging
PM2.5 data is organized by hour, but we need it on a daily level so it can map in with the AQI levels. First we do a groupby to get daily levels, then we need to remerge with the original data to get the categorical data back in (county, site, lat, lon)

In [121]:
df_sample1 = df_sample[['site','sample','month_year','cbsa_code']].groupby(['month_year','cbsa_code','site']).mean().reset_index().sort_values('month_year',ascending = False)
df_sample1.head()
len(df_sample1)
print(df_sample1.head(3))
print('-----------------------------------------')
print('There are '+ str(len(df_sample1)) + ' rows after grouping')

     month_year cbsa_code  site    sample
2508    12-2015     49700  0003  9.771429
2453    12-2015     17140  3002  6.790000
2456    12-2015     17460  0034  8.100000
-----------------------------------------
There are 2509 rows after grouping


# Same process for PM2.5 now for AQI
AQI = Air Quality Index
This is not done every day but every 1 - 3 days depending on how the EPA decided to track it.

In [37]:
response_daily = []
start = ["20150101","20140101","20130101"]
end = ["20151231","2014231","20131231"]
codes = msa_codes['CBSA Code']

for index in range(len(start)):
    for each_msa in codes:
        url = f"https://aqs.epa.gov/data/api/dailyData/byCBSA?email={api_id}&key={api_key}&param=88101&bdate={start[index]}&edate={end[index]}&cbsa={each_msa}"
        response_daily.append(requests.get(url).json())

In [38]:
date = []
cbsa_code = []
site = []
aqi = []


for x in range(len(response_daily)):
    for y in range(0,response_daily[x]['Header'][0]['rows']):
        date.append(response_daily[x]['Data'][y]['date_local'])
        aqi.append(response_daily[x]['Data'][y]['aqi'])
        cbsa_code.append(response_daily[x]['Data'][y]['cbsa_code'])
        site.append(response_daily[x]['Data'][y]['site_number'])

        
columns = ['date','aqi','cbsa_code','site']
df_daily = pd.DataFrame(data = list(zip(date,aqi,cbsa_code,site)), columns = columns).drop_duplicates().dropna()
df_daily.head()

KeyError: 'rows'

In [ ]:
print(df_daily.head())
print('---------------------------------------------------------')
print('There are '+ str(len(df_daily)) + ' rows from the API pull')

In [ ]:
df_daily['date'] = pd.to_datetime(df_daily.date,format = '%Y-%m')
df_daily.head(1)

In [ ]:
df_daily['month_year'] = df_daily['date'].dt.strftime('%m-%Y')
df_daily.head(1)

In [ ]:
df_daily1 = df_daily[['site','aqi','month_year','cbsa_code']].groupby(['month_year','cbsa_code','site']).mean().reset_index().sort_values('month_year',ascending = False)
df_daily1.head()

# API Call Census Data
pulling on 5 polluted state/counties from random selection of the top 20 highest polluted counties in the US

pulling sectors for mining/quarring, utilities, construction, manufactoring, & wholesale trade

https://classcodes.com/naics-2-digit-sector-codes/

In [ ]:
year_survey = ['2017','2015','2014']
year_actual = ['2015','2014','2013']
variables_interest = ['NAICS2012_TTL,EMP,ESTAB']
sectors = ["31-33","21","22","42","48-49"]
# sectors =["31-33"]
codes = msa_codes['CBSA Code']
response_census = []
year_date = []


for index in range(len(year)):
    for each_msa in codes:
        try:
            base_url = f"https://api.census.gov/data/{year_survey[index]}/cbp?get={variables_interest[0]}&NAICS2012={sectors[0]}&NAICS2012={sectors[1]}&NAICS2012={sectors[2]}&NAICS2012={sectors[3]}&NAICS2012={sectors[4]}&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:{each_msa}&key={api_key2}"
            response_census.append(requests.get(base_url).json())
            year_date.append(year_actual[index])
        except:
            print(f'MSA {each_msa} not found')

In [ ]:
response_census

In [ ]:
len(response_census)

In [ ]:
naics2012_ttl = []
cbsa_code = []
emp = []
estab = []
sector = []
date = []

for x in range(len(response_census)):
    for y in range(1,5):
        sector.append(response_census[x][y][0])
        cbsa_code.append(response_census[x][y][4])
        emp.append(response_census[x][y][1])
        estab.append(response_census[x][y][2]) 

# Creating Census Dataset from Pulled Data
dropping first index as it is the column heading

In [ ]:
columns = ['sector','cbsa_code','emp','estab','year']
census = pd.DataFrame(data = list(zip(sector,cbsa_code,emp,estab,year_date)), columns = columns)
census

# Creating Dummy Variables for Categorical Data of Industries and CBSA Codes

In [ ]:
df2 = pd.get_dummies(census['sector'])
df2.head()

In [ ]:
df3 = pd.get_dummies(census['cbsa_code'])
df3

# Concatenating Dummy Variables back in

In [ ]:
census_final = pd.concat([df2,df3,census],axis = 1).dropna()
census_final.head()

In [ ]:
census_final= census_final.astype({'emp': float,
                                  'estab':float})
census_final.head()

# Reading in and Cleaning Oil Data

In [ ]:
#remove rows before 198
file = "../data/Crude_oil_prices.csv"
oil = pd.read_csv(file)
oil_data=oil[(oil['Year']>1984)]
oil_data['date'] = pd.to_datetime(oil_data[['Year', 'Month']].assign(DAY=1))
oil_data= oil_data.drop(columns=['Free on Board Cost of Crude Oil Imports (Dollars per Barrel)',
                                 'Landed Cost of Crude Oil Imports (Dollars per Barrel)',
                                 'Refiner Acquisition Cost of Crude Oil, Domestic (Dollars per Barrel)',
                                 'Refiner Acquisition Cost of Crude Oil, Imported (Dollars per Barrel)',
                                 'Refiner Acquisition Cost of Crude Oil, Composite (Dollars per Barrel)','Month','Year'])
oil_data.head()

In [ ]:
oil_data= oil_data.rename(columns={"Crude Oil Domestic First Purchase Price (Dollars per Barrel)":'Crude_Oil_Price'})
oil_data= oil_data[['date','Crude_Oil_Price']]
index= oil_data[(oil_data['Crude_Oil_Price'] =='Not Available')].index

oil_data.drop(index, inplace=True)

oil_data.head()

In [ ]:
oil_data['month_year'] = oil_data['date'].dt.strftime('%m-%Y')
oil_data.head()

In [ ]:
oil_data= oil_data.astype({'Crude_Oil_Price': float})
oil_data.head()

In [ ]:
oil_data1 = oil_data.groupby('month_year').mean().reset_index()
oil_data1.head()

# Merging in Oil Prices into Census & Pollution Data

In [ ]:
pm25_census = pd.merge(df_sample1, census_final,
                      how = 'inner',
                      on = 'cbsa_code')
pm25_census.head()

In [ ]:
pm25_census_oil = pd.merge(pm25_census, oil_data1,
                          how = 'inner',
                          on = 'month_year')
pm25_census_oil

In [ ]:
aqi_census = pd.merge(df_daily1,census_final,
                     how = 'inner',
                     on = 'cbsa_code')
aqi_census.head()

In [ ]:
aqi_census_oil = pd.merge(aqi_census, oil_data1,
                          how = 'inner',
                          on = 'month_year')
aqi_census_oil.head()

In [ ]:
y = aqi_census_oil['aqi']
x = aqi_census_oil[['Crude_Oil_Price','Manufacturing','Mining, quarrying, and oil and gas extraction','Transportation and warehousing','Utilities','emp','estab']]

In [ ]:
model = sm.OLS(y,x).fit()
predictions = model.predict(x)
plt.rc('figure', figsize=(12, 7))
plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 12}, fontproperties = 'monospace')
plt.axis('off')
plt.tight_layout()
plt.show()

# Exporting AQI, Census, Oil Data to CSV to use in Tableau

In [ ]:
aqi_census_oil.to_csv('../data/aqi_census_oil1.csv')


In [ ]:
y1 = pm25_census_oil['sample']
x1 = pm25_census_oil[['Crude_Oil_Price','Manufacturing','Mining, quarrying, and oil and gas extraction','Transportation and warehousing','Utilities']]

In [ ]:
model = sm.OLS(y1,x1).fit()
predictions = model.predict(x1)
plt.rc('figure', figsize=(12, 7))
plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 12}, fontproperties = 'monospace')
plt.axis('off')
plt.tight_layout()
plt.show()